# Assignment Instruction

This is part 2 of assignment 1. It amounts to 60 percent of the grade.

In this part of the assignment, you will explore covid-19 data from the US. 

The dataset is:

1. At the county level
2. Including deaths and case figures
3. On 1 Oct 2020.

"This dataset contains daily updated number of reported cases & deaths in the US on the state and county level, as provided by the Johns Hopkins University." It is downloaded from https://www.kaggle.com/headsortails/covid19-us-county-jhu-data-demographics

In addition, you will use two additional datasets regarding the demography of the counties and political factors.

1. County demography
    - https://www.kaggle.com/muonneutrino/us-census-demographic-data
2. The county-level outcomes of the 2016 US Presidential Election
    - https://www.kaggle.com/stevepalley/2016uspresidentialvotebycounty

The data files are found in `Data` directory of this repository. You can download the files, and upload.





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import packages

In [ ]:
import numpy as np
import pandas as pd

# Covid data (15 percent)

## Read data

Open `covid_us_county_oct20.csv`, and print the first 10 rows

In [ ]:
df = pd.read_csv("/content/drive/My Drive/data_gv918/Data/covid_us_county_oct20.csv")
df.head(10)

## Answer the following questions

1. How many rows in the dataset?
2. What's the highest count of cases? In which county?
3. Count the number of counties in each US State
4. Calculate the following numbers by US States
    - Cases
    - Deaths

In [ ]:
df.shape

In [ ]:
df['cases'].max()


In [ ]:
df[df['cases'] == df['cases'].max()]

In [ ]:

df['state'].value_counts()

In [ ]:
df[df['fips'] < 80000]['state'].value_counts()

In [ ]:
df[['state', 'deaths', 'cases']].groupby(by = 'state').sum().sort_values(by = 'deaths', ascending = False)

# Working with the demography data (20 percent)


The data (`acs2017_county_data.csv`) is coming from https://www.kaggle.com/muonneutrino/us-census-demographic-data

Read the data description on the website, and answer the following. 




## Read the data

- Read the data and print the first 5 rows

In [ ]:
df_demo = pd.read_csv("/content/drive/My Drive/data_gv918/Data/acs2017_county_data.csv")

df_demo.head(5)

## Explore the data

- How many rows in the data?
- Which column do you use to merge this demography data with Covid-19 data?
- Which columns to drop from the demography dataset (these are potentially duplicated in covid 19)? 

In [ ]:
df_demo.shape

In [ ]:
df_demo.rename(columns = {'CountyId': 'fips'}, inplace=True)

In [ ]:
df_demo.drop(['State', 'County'], axis = 1, inplace = True)

## Merge the data

- Merge two data frames
- Answer the following questions
  - Calculate the deaths/cases per 1000 population
  - Find the maximum of highest deaths/cases per 1000 population and their county names 
- Aggregate the data at the state level and find the following
  - Calculate the number of total cases/deaths per 1000 population
  - Which state had the highest cases/deaths, how many cases/deaths?


In [ ]:
df_merged = df.merge(df_demo, on = 'fips')
df_merged.head()

In [ ]:
df_merged['case_1k'] = df_merged['cases'] / df_merged['TotalPop'] * 1000
df_merged['death_1k'] = df_merged['deaths'] / df_merged['TotalPop'] * 1000


In [ ]:
df_merged.sort_values('case_1k', ascending=False).head(1)


In [ ]:
df_merged.sort_values('death_1k', ascending=False).head(1)

## Data exploration

- Limit the data to the counties with more than 1000 people 
- Several variables are expected to be correlated with the high contraction rate of Covid-19, such as the racial composition of the county or household income. Check the correlation between these variables and cases/deaths rates. What do you find? Do you find any strong correlations?


In [ ]:
df_merged_sub = df_merged[df_merged['TotalPop'] > 1000]
df_merged_sub.shape

In [ ]:
df_merged_sub[['case_1k', 'death_1k', 'White', 'Black', 'IncomePerCap', 'Poverty']].corr()

# Presidential voting in 2016 (15 percent)

It is well-known that there is a partisan divide in the US citizens' perceptions about the risk of Covid-19 and their response to it (e.g. https://www.pnas.org/content/117/39/24144, but there are numerous academic/journalistic articles you can find). In this section, you will check whether the votes in the 2016 US Presidential Election are correlated with the case/death figures.

The data (`pres16results.csv`) is coming from https://www.kaggle.com/stevepalley/2016uspresidentialvotebycounty




## Read the data and reshape/filter

- Read the data
- Reshape/filter the data
    - this data has a long format
    - you need to conduct data manipulation before the merging
    - you can either drop some rows (e.g. keep only Trump rows) or make it to the wide format


In [ ]:
df_pres = pd.read_csv("/content/drive/My Drive/data_gv918/Data/pres16results.csv")
#df_pres.head()

In [ ]:
df_pres_trump = df_pres[df_pres['cand'] == 'Donald Trump'][['fips','pct']].rename({'pct':'pct_trump'}, axis = 1)

df_pres_trump['fips'] = pd.to_numeric(df_pres_trump['fips'], errors = 'coerce')
df_pres_trump.dropna()

## Merge the data and explore the relations

- Merge the presidential election results with the main data
- Explore the relations between Trump (or Clinton) vote percentage and Covid-19 variables. What do you find?

In [ ]:
df_full = df_merged.merge(df_pres_trump, on = 'fips')
df_full.head()

In [ ]:
df_full.shape

In [ ]:
df_full[['case_1k', 'death_1k', 'White', 'Black', 'IncomePerCap', 'Poverty', 'pct_trump']].corr()

# Summary (10 percent)

- Summarise your finding in (about) 300-600 words
- You can discuss the following (or you can just be creative)
    - Did you find any strong correlation between the demography/political variables with the case/death figures?
    - If you did, what do you find? Did it match with the common perception about who is more likely to contract the disease?
    - If not, why you think you didn't? Is it because of some data issues? Or something else?